# <center>Case Study of loans_full_schema Dataset</center>

Miten Dalsaniya

<br>

In [1]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')

## Load Data 

In [2]:
data = pd.read_csv("loans_full_schema.csv")

## Describe Data

In [3]:
data.describe(include='all')

,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,delinq_2y,months_since_last_delinq,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,num_collections_last_12m,num_historical_failed_to_pay,months_since_90d_late,current_accounts_delinq,total_collection_amount_ever,current_installment_accounts,accounts_opened_24m,months_since_last_credit_inquiry,num_satisfactory_accounts,num_accounts_120d_past_due,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,tax_liens,public_record_bankrupt,loan_purpose,application_type,loan_amount,term,interest_rate,installment,grade,sub_grade,issue_month,loan_status,initial_listing_status,disbursement_method,balance,paid_total,paid_principal,paid_interest,paid_late_fees
count,9167,9183.000000,10000,10000,1.000000e+04,10000,9976.000000,1.495000e+03,1455,1495.000000,10000.00000,4342.000000,10000.00000,10000.00000,10000.000000,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,2285.000000,10000.0000,10000.000000,10000.000000,10000.000000,8729.000000,10000.000000,9682.0,10000.0000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000,10000,10000,10000,10000,10000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
unique,4741,NaN,50,3,NaN,3,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,2,NaN,NaN,NaN,NaN,7,32,3,6,2,2,NaN,NaN,NaN,NaN,NaN
top,manager,NaN,CA,MORTGAGE,NaN,Source Verified,NaN,NaN,Not Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,debt_consolidation,individual,NaN,NaN,NaN,NaN,B,B1,Mar-2018,Current,whole,Cash,NaN,NaN,NaN,NaN,NaN
freq,218,NaN,1330,4789,NaN,4116,NaN,NaN,611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5144,8505,NaN,NaN,NaN,NaN,3037,647,3617,9375,8206,9284,NaN,NaN,NaN,NaN,NaN
mean,NaN,5.930306,NaN,NaN,7.922215e+04,NaN,19.308192,1.279146e+05,NaN,19.979304,0.21600,36.760709,2001.29000,1.95820,22.679600,11.40440,1.836062e+05,51049.063100,0.013800,0.167100,46.112473,0.0001,184.334000,2.663600,4.376100,7.340703,11.379600,0.0,0.0001,3.595300,27357.410300,13.030000,8.095000,5.230500,1.383100,94.648990,0.043300,0.123800,NaN,NaN,16361.922500,43.272000,12.427524,476.205323,NaN,NaN,NaN,NaN,NaN,NaN,14458.916610,2494.234773,1894.448466,599.666781,0.119516
std,NaN,3.703734,NaN,NaN,6.473429e+04,NaN,15.004851,7.016838e+04,NaN,8.054781,0.68366,21.634939,7.79551,2.38013,11.885439,5.86828,1.876327e+05,53636.731172,0.126535,0.687768,21.748021,0.0100,2206.389859,2.935387,3.158583,5.981590,5.859705,0.0,0.0100,2.434689,26570.164081,7.874713,4.906606,3.327623,1.717274,9.155059,0.593852,0.337172,NaN,NaN,10301.956759,11.029877,5.001105,294.851627,NaN,NaN,NaN,NaN,NaN,NaN,9964.561865,3958.230365,3884.407175,517.328062,1.813468
min,NaN,0.000000,NaN,NaN,0.000000e+00,NaN,0.000000,1.920000e+04,NaN,0.320000,0.00000,1.000000,1963.00000,0.00000,2.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000,2.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,14.300000,0.000000,0.000000,NaN,NaN,1000.000000,36.000000,5.310000,30.750000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,2.000000,NaN,NaN,4.500000e+04,NaN,11.057500,8.683350e+04,NaN,14.160000,0.00000,19.000000,1997.00000,0.00000,14.000000,7.00000,5.159375e+04,19185.500000,0.000000,0.000000,29.000000,0.0000,0.000000,1.000000,2.000000,2.000000,7.000000,0.0,0.0000,2.000000,10000.000000,7.

In [4]:
data.head()

,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,delinq_2y,months_since_last_delinq,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,num_collections_last_12m,num_historical_failed_to_pay,months_since_90d_late,current_accounts_delinq,total_collection_amount_ever,current_installment_accounts,accounts_opened_24m,months_since_last_credit_inquiry,num_satisfactory_accounts,num_accounts_120d_past_due,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,tax_liens,public_record_bankrupt,loan_purpose,application_type,loan_amount,term,interest_rate,installment,grade,sub_grade,issue_month,loan_status,initial_listing_status,disbursement_method,balance,paid_total,paid_principal,paid_interest,paid_late_fees
0,global config engineer,3.0,NJ,MORTGAGE,90000.0,Verified,18.01,NaN,NaN,NaN,0,38.0,2001,6,28,10,70795,38767,0,0,38.0,0,1250,2,5,5.0,10,0.0,0,2,11100,14,8,6,1,92.9,0,0,moving,individual,28000,60,14.07,652.53,C,C3,Mar-2018,Current,whole,Cash,27015.86,1999.33,984.14,1015.19,0.0
1,warehouse office clerk,10.0,HI,RENT,40000.0,Not Verified,5.04,NaN,NaN,NaN,0,NaN,1996,1,30,14,28800,4321,0,1,NaN,0,0,0,11,8.0,14,0.0,0,3,16500,24,14,4,0,100.0,0,1,debt_consolidation,individual,5000,36,12.61,167.54,C,C1,Feb-2018,Current,whole,Cash,4651.37,499.12,348.63,150.49,0.0
2,assembly,3.0,WI,RENT,40000.0,Source Verified,21.15,NaN,NaN,NaN,0,28.0,2006,4,31,10,24193,16000,0,0,28.0,0,432,1,13,7.0,10,0.0,0,3,4300,14,8,6,0,93.5,0,0,other,individual,2000,36,17.09,71.40,D,D1,Feb-2018,Current,fractional,Cash,1824.63,281.80,175.37,106.43,0.0
3,customer service,1.0,PA,RENT,30000.0,Not Verified,10.16,NaN,NaN,NaN,0,NaN,2007,0,4,4,25400,4997,0,1,NaN,0,0,1,1,15.0,4,0.0,0,2,19400,3,3,2,0,100.0,1,0,debt_consolidation,individual,21600,36,6.72,664.19,A,A3,Jan-2018,Current,whole,Cash,18853.26,3312.89,2746.74,566.15,0.0
4,security supervisor,10.0,CA,RENT,35000.0,Verified,57.96,57000.0,Verified,37.66,0,NaN,2008,7,22,16,69839,52722,0,0,NaN,0,0,1,6,4.0,16,0.0,0,10,32700,20,15,13,0,100.0,0,0,credit_card,joint,23000,36,14.07,786.87,C,C3,Mar-2018,Current,whole,Cash,21430.15,2324.65,1569.85,754.80,0.0


In [5]:
data.shape

(10000, 55)

In [6]:
data.isnull().sum()

emp_title                            833
emp_length                           817
state                                  0
homeownership                          0
annual_income                          0
verified_income                        0
debt_to_income                        24
annual_income_joint                 8505
verification_income_joint           8545
debt_to_income_joint                8505
delinq_2y                              0
months_since_last_delinq            5658
earliest_credit_line                   0
inquiries_last_12m                     0
total_credit_lines                     0
open_credit_lines                      0
total_credit_limit                     0
total_credit_utilized                  0
num_collections_last_12m               0
num_historical_failed_to_pay           0
months_since_90d_late               7715
current_accounts_delinq                0
total_collection_amount_ever           0
current_installment_accounts           0
accounts_opened_

In [7]:
data.dtypes

emp_title                            object
emp_length                          float64
state                                object
homeownership                        object
annual_income                       float64
verified_income                      object
debt_to_income                      float64
annual_income_joint                 float64
verification_income_joint            object
debt_to_income_joint                float64
delinq_2y                             int64
months_since_last_delinq            float64
earliest_credit_line                  int64
inquiries_last_12m                    int64
total_credit_lines                    int64
open_credit_lines                     int64
total_credit_limit                    int64
total_credit_utilized                 int64
num_collections_last_12m              int64
num_historical_failed_to_pay          int64
months_since_90d_late               float64
current_accounts_delinq               int64
total_collection_amount_ever    

The dataset has 10,000 rows and 55 columns. For the 55 columns, 42 of them are either float or int type and the rest are object.Two issues I found are:
1. Some columns have high percentage of missing value, such as debt_to_income, annual_income_joint, verification_income_joint, debt_to_income_joint and months_since_90d_late.
2. The data type for some columns are not unified or appropriate. For example, the data type for "num_accounts_120d_past_due" is float64, while the data type for "num_accounts_30d_past_due" is int64. Also, the data type for columns related to month or year is float64 while the data type for columns related to num or total is int64. I thinkthis should be reversed.

To predict the interest_rate, I used multiple linear regression and neural network because they are good for predicting continuous variables.

### Multiple Linear Regression

Assumptions:<br> 
1.The relationship between x and the mean of y is linear.<br> 
2.The variance of residual is the same for any value of x.<br> 
3.Observations are independent of each other.<br> 
4.For any fixed value of x, y is normally distributed.

#### Step 1: Set up the dependent and the independent variables

Dependent variables: 'interest_rate'<br> 

Independent variables: 'inquiries_last_12m','total_credit_lines','open_credit_lines','total_credit_limit','total_credit_utilized','num_collections_last_12m',<br> 'num_historical_failed_to_pay','current_accounts_delinq','total_collection_amount_ever','current_installment_accounts','accounts_opened_24m',<br> 'num_satisfactory_accounts','num_accounts_30d_past_due','num_active_debit_accounts','total_debit_limit','num_total_cc_accounts',<br> 'num_open_cc_accounts','num_cc_carrying_balance','num_mort_accounts','account_never_delinq_percent','tax_liens','public_record_bankrupt'

In [8]:
x = data[['inquiries_last_12m','total_credit_lines','open_credit_lines','total_credit_limit','total_credit_utilized','num_collections_last_12m','num_historical_failed_to_pay','current_accounts_delinq','total_collection_amount_ever','current_installment_accounts','accounts_opened_24m','num_satisfactory_accounts','num_accounts_30d_past_due','num_active_debit_accounts','total_debit_limit','num_total_cc_accounts','num_open_cc_accounts','num_cc_carrying_balance','num_mort_accounts','account_never_delinq_percent','tax_liens','public_record_bankrupt']].astype("float")
y = data[['interest_rate']].astype("float")
x.dtypes

inquiries_last_12m              float64
total_credit_lines              float64
open_credit_lines               float64
total_credit_limit              float64
total_credit_utilized           float64
num_collections_last_12m        float64
num_historical_failed_to_pay    float64
current_accounts_delinq         float64
total_collection_amount_ever    float64
current_installment_accounts    float64
accounts_opened_24m             float64
num_satisfactory_accounts       float64
num_accounts_30d_past_due       float64
num_active_debit_accounts       float64
total_debit_limit               float64
num_total_cc_accounts           float64
num_open_cc_accounts            float64
num_cc_carrying_balance         float64
num_mort_accounts               float64
account_never_delinq_percent    float64
tax_liens                       float64
public_record_bankrupt          float64
dtype: object

#### Step 2: Divide the data into train and test sets

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 5)

In [10]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 22)
(2000, 22)
(8000, 1)
(2000, 1)


#### Step 3: Fit the model

In [14]:
regressor = LinearRegression()
regressor.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
v = pd.DataFrame(regressor.coef_,index=["Co-efficient"]).transpose()
w = pd.DataFrame(x.columns,columns = ["Attribute"])

In [16]:
coeff_df = pd.concat([w,v],axis=1,join="inner").sort_values(by="Co-efficient",ascending=False,key=abs).reset_index(drop=True) 
coeff_df

,Attribute,Co-efficient
0,current_accounts_delinq,1.758619
1,num_accounts_30d_past_due,1.758619
2,num_collections_last_12m,0.862531
3,open_credit_lines,0.299650
4,num_cc_carrying_balance,0.283276
5,num_satisfactory_accounts,-0.282719
6,num_mort_accounts,-0.215976
7,inquiries_last_12m,0.201500
8,accounts_opened_24m,0.181791
9,num_active_debit_accounts,0.138182


We can find that some independent variables which have high corelation with dependent variable include num_accounts_30d_past_due, current_accounts_delinq,num_collections_last_12.

#### Step 4: Make predictions on test data

In [17]:
y_pred = regressor.predict(x_test)
y_pred = pd.DataFrame(y_pred,columns=["Predicted"])
y_pred

,Predicted
0,11.310512
1,12.195906
2,11.771405
3,12.692099
4,9.048018
...,...
1995,13.471518
1996,11.152936
1997,0.007554
1998,11.492837


#### Step 5: Evaluate the algorithm

In [18]:
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test,y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

Mean Absolute Error: 3.626854833713241
Mean Squared Error: 21.492793553030133
Root Mean Squared Error: 4.6360320914581825


Enhancements if have more time: <br> 
1.use scatterplot to check the linearity before choosing the independent variables.<br> 
2.check for outlier.<br> 
3.compare train error and test error to see whether there is overfitting.<br> 
4.cross-validate the data to check how well the model fits to data that it has not been trained on. 

### Neural Network

#### Step 1: Split the data

In [19]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x, y, test_size = 0.2, random_state = 5)

#### Step 2: Scale the Data

The neural network in Python may have difficulty converging before the maximum number of iterations allowed if the data is not normalized, so we scale the data first. 

In [20]:
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(x1_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
# Now apply the transformations to the data:
x1_train = scaler.transform(x1_train)
x1_test = scaler.transform(x1_test)

#### Step 3: Train the Model

In [21]:
x1_train, x1_test, y1_train, y1_test = x1_train.astype('int'), x1_test.astype('int'), y1_train.astype('int'), y1_test.astype('int')
mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
mlp.fit(x1_train,y1_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(8, 8, 8), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

#### Step 4: Make predictions on test data

In [22]:
y1_pred = mlp.predict(x1_test)
y1_pred = pd.DataFrame(y1_pred,columns=["Predicted"])
y1_pred

,Predicted
0,6
1,6
2,9
3,10
4,6
...,...
1995,9
1996,9
1997,5
1998,9


#### Step 5: Evaluate the algorithm

In [23]:
print("Mean Absolute Error:", metrics.mean_absolute_error(y1_test,y1_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y1_test,y1_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y1_test,y1_pred)))

Mean Absolute Error: 4.3205
Mean Squared Error: 35.2495
Root Mean Squared Error: 5.937128935773586


By comparing the evaluation of multiple linear regression and neural network, <br> we can find that multiple linear regression have a better performance for all three metrics.

Enhancements if have more time:<br>
1.cross-validate the data to check how well the model fits to data that it has not been trained on.<br>
2.check for outlier.<br>
3.compare train error and test error to see whether there is overfitting.<br>